In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Resizing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from drive.MyDrive.DL.codes.transcription.utils import plot_training
import tensorflow as tf
from drive.MyDrive.DL.codes.hatespeech.pipelines import ML,binarize, vectorize
import warnings
warnings.filterwarnings('ignore')
obj = 'txt'
early_stopping = EarlyStopping(monitor='loss', patience=5)

In [ ]:
train = tf.keras.utils.image_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/train',labels='inferred',label_mode='int',color_mode='rgb',batch_size=32,image_size=(224,224))
valid = tf.keras.utils.image_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/val',labels='inferred',label_mode='int',color_mode='rgb',batch_size=32,image_size=(224,224))
test  = tf.keras.utils.image_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/test',labels='inferred',label_mode='int',color_mode='rgb',batch_size=32,image_size=(224,224))

train = train.map(binarize)
valid = valid.map(binarize)
test = test.map(binarize)

resnet_model = ResNet50(include_top=False,input_shape=(224,224,3))
x = Flatten()(resnet_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=resnet_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    train,
    validation_data=valid,
    epochs=150,
    callbacks=[early_stopping]
)

model.evaluate(test)
plot_training(history)

Found 71496 files belonging to 6 classes.
Found 5000 files belonging to 6 classes.
Found 10000 files belonging to 6 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 4625s 2s/step - accuracy: 0.6260 - loss: 0.9700 - val_accuracy: 0.5510 - val_loss: 0.6991
Epoch 2/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 122s 54ms/step - accuracy: 0.6313 - loss: 0.6609 - val_accuracy: 0.5510 - val_loss: 0.7046
Epoch 3/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 121s 54ms/step - accuracy: 0.6303 - loss: 0.6726 - val_accuracy: 0.5510 - val_loss: 0.6987
Epoch 4/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 121s 54ms/step - accuracy: 0.6308 - loss: 0.6611 - val_accuracy: 0.5516 - val_loss: 0.7020
Epoch 5/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 121s 54ms/step - accuracy: 0.6304 - loss: 0.6603 - val_accuracy: 0.5510 - val_loss: 0.7021
Epoch 6/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 121s 54ms/step - accuracy: 0.6307 - loss: 0.6590 - val_accuracy: 0.5524 - val_loss: 0.7156
Epoch 7/150
2235/2235

In [ ]:
train = tf.keras.utils.text_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/train',labels='inferred',label_mode='int',batch_size=32)
valid = tf.keras.utils.text_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/val',labels='inferred',label_mode='int',batch_size=32)
test  = tf.keras.utils.text_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/test',labels='inferred',label_mode='int',batch_size=32)

train = train.map(binarize)
valid = valid.map(binarize)
test = test.map(binarize)

train = train.map(vectorize)
valid = valid.map(vectorize)
test = test.map(vectorize)

resnet_model = ResNet50(include_top=False,input_shape=(224,224,1))

x = Resizing(224, 224,1)(resnet_model.input)
x = resnet_model(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=resnet_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    train,
    validation_data=valid,
    epochs=150,
    callbacks=[early_stopping]
)

model.evaluate(test)
plot_training(history)